# Hypothesis Testing in Healthcare: Drug Safety 

A pharmaceutical company GlobalXYZ has just completed a randomized controlled drug trial. To promote transparency and reproducibility of the drug's outcome, they (GlobalXYZ) have presented the dataset to your organization, a non-profit that focuses primarily on drug safety.

The dataset provided contained five adverse effects, demographic data, vital signs, etc. Your organization is primarily interested in the drug's adverse reactions. It wants to know if the adverse reactions, if any, are of significant proportions. It has asked you to explore and answer some questions from the data.

The dataset drug_safety.csv was obtained from https://hbiostat.org/data/ courtesy of the Vanderbilt University Department of Biostatistics. It contained five adverse effects: headache, abdominal pain, dyspepsia, upper respiratory infection, chronic obstructive airway disease (COAD), demographic data, vital signs, lab measures, etc. The ratio of drug observations to placebo observations is 2 to 1.

The purpose of this study is to:

Determine if the proportion of adverse effects differs significantly between the Drug and Placebo groups.

Find out if the number of adverse effects is independent of the treatment and control groups, saving as a variable called num_effects_p_value containing a p-value.

Examine if there is a significant difference between the ages of the

In [5]:
pip install pyreadr

  Obtaining dependency information for pyreadr from https://files.pythonhosted.org/packages/cf/47/1f666551444156b18524632fb93da10bbcf9a9cf0cb20faebd1f85512558/pyreadr-0.5.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 2.8 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pyreadr

safety_data = pyreadr.read_r('safety.rda')

df = safety_data['All']
df.head()

,week,headache,ab.pain,dyspepsia,upper.resp.infect,coad,trx,age,sex,race,...,pr,qrs,rr,uncorr.qt,dbp,hr,sbp,id,nausea,diarrhea
rownames,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,Drug,62,male,Caucasian,...,156,64,571.400024,340,80,88,128,1,0,0
2,1,0,0,0,0,0,Drug,62,male,Caucasian,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
3,12,0,0,0,0,0,Drug,62,male,Caucasian,...,156,76,722.799988,368,78,88,120,1,0,0
4,16,0,0,0,0,0,Drug,62,male,Caucasian,...,160,72,689.599976,360,72,88,126,1,0,0
5,2,0,0,0,0,0,Drug,62,male,Caucasian,...,160,72,666.599976,352,82,86,120,1,0,0


To make more easy to read this information, we are going to manipulate the data base to get the next order: 

| Column | Description |
|--------|-------------|
|sex | The gender of the individual|
| age | The age of the individual|
| week |The week of the drug testing|
|trx |The treatment (Drug) and control (Placebo) groups
|wbc | The count of white blood cells
|rbc |The count of red blood cells
|adverse_effects |The presence of at least a single adverse effect
|num_effects| The number of adverse effects experienced by a single individual

In [58]:
import pandas as pd

order = ['sex', 'age', 'week', 'trx', 'wbc', 'rbc','adverse_effects','num_effects']
df['adverse_effects'] = df[['headache','ab.pain','dyspepsia','upper.resp.infect','coad']].any(axis=1)
df['num_effects'] = df[['headache','ab.pain','dyspepsia','upper.resp.infect','coad']].sum(axis=1)
df2 = df[order]
df2.head()

,sex,age,week,trx,wbc,rbc,adverse_effects,num_effects
rownames,,,,,,,,
1,male,62,0,Drug,7.3,5.1,False,0
2,male,62,1,Drug,NaN,NaN,False,0
3,male,62,12,Drug,5.6,5.0,False,0
4,male,62,16,Drug,NaN,NaN,False,0
5,male,62,2,Drug,6.6,5.1,False,0


# The proportion of adverse effects differs significantly between the Drug and Placebo groups?

To answer this question, I'm going to compare the proportion of participants with adverse effects in two groups:

- Drug
- Placebo

And make two different hypothesis:

- **Null Hypothesis:** the proportion of adverse effects is equal in both groups
- **Alternative Hypothesis:** the proportion of adverse effects differs between each group

For this I'll start creating an contingency table:

In [60]:
contingency_table = pd.crosstab(df['trx'],df['adverse_effects'])
contingency_table

adverse_effects,False,True
trx,,
Drug,9968,1024
Placebo,4960,512
